In [1]:
import awkward as ak
import numpy as np
import uproot as up
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import matplotlib.pyplot as plt
import correctionlib
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory
from coffea.lookup_tools import extractor

fname = "/home/saikat/analysis/MC_files/ZG2JtoG2L2J_EWK_MLL-50_MJJ-120_TuneCP5_withDipoleRecoil_13p6TeV_madgraph-pythia8/ZG2JtoG2L2J_EWK_MLL-50_MJJ-120_TuneCP5_withDipoleRecoil_13p6TeV_madgraph-pythia8.root"
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "ZGammaToLLGamma"},
).events()
totGenWeight = np.sum(events.genWeight)
totGenWeight

698608.0

# Working with $\mu$s

In [5]:
#Applying Trigger cuts 
hlt = events.HLT
#hlt.fields
trig_cut = np.stack((hlt.IsoMu24, hlt.IsoMu27), axis=1)
events = events[ak.any(trig_cut, axis=1)]

In [13]:
muon = events.Muon
muon.fields
muon_sel = (ak.num(muon)>=2) & (abs(muon.eta)<2.4) & (muon.tightId==1) & (abs(muon.dxy)<0.2) & (abs(muon.dz)<0.2)
muon = muon[muon_sel]
muon = muon[ak.num(muon)==2]
events = events[ak.num(muon)==2]
muon.pt

<Array [[76.8, 26.9], [134, ... [128, 77.5]] type='152922 * var * float32[parame...'>